In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import statistics as st

# from statsmodels.formula.api import ols
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import statsmodels.api as sm
# import scipy.stats as stats
# from sklearn.model_selection import train_test_split

#### Read in data, set each DF to a variable; then split the data into a Train and Test set (80/20).

#### This allows for 'wild' data simulation i.e. the model doesn't get trained on the actual test set at all

- fertilizers = pd.read_csv('Data/fao_data_fertilizers_data.csv')
- fert_train, fert_test = train_test_split(fertilizers)

- forest = pd.read_csv('Data/fao_data_forest_data.csv')
- forest_train, forest_test = train_test_split(forest)

- land = pd.read_csv('Data/fao_data_land_data.csv')
- land_train, land_test = train_test_split(land)

- production = pd.read_csv('Data/fao_data_production_indices_data.csv')
- prod_train, prod_test = train_test_split(production)

#### Print train & test shape
- print('TRAIN SHAPE:')
- print('fert: ', fert_train.shape, 'forest: ', forest_train.shape, 'land: ', land_train.shape, 'prod: ', prod_train.shape)
- print()
- print('TEST SHAPE:')
- print('fert: ', fert_test.shape, 'forest: ', forest_test.shape, 'land: ', land_test.shape, 'prod: ', prod_test.shape)

### Save files for future work
- prod_train.to_csv('Data/Model Data/prod_train.csv', index=False)
- prod_test.to_csv('Data/Model Data/prod_test.csv', index=False)

# Read in train & test files

In [2]:
prod_train = pd.read_csv('Data/Model Data/prod_train.csv')
prod_test = pd.read_csv('Data/Model Data/prod_test.csv')

# Investigating Production Data Structure

In [3]:
# Change the name of the dataframe for ease in coding
df = prod_train

# Change the name of the columns first
headers = ["Region", "Element Code", "Element", "Year", "Unit", "Dollar Amount", "Value Footnotes", "Category"]
df.columns = headers

# Figure out what we're replacing null values with for each column
df_shape_og = df.shape
print(df.isna().sum())
df.head()

Region             0
Element Code       0
Element            9
Year               9
Unit               9
Dollar Amount      9
Value Footnotes    9
Category           0
dtype: int64


,Region,Element Code,Element,Year,Unit,Dollar Amount,Value Footnotes,Category
0,Caribbean +,438,Net per capita PIN (base 1999-2001),1977.0,Int. $,115.0,A,livestock_pin
1,Kiribati,434,Grs per capita PIN (base 1999-2001),1980.0,Int. $,102.0,A,crops_pin
2,Micronesia +,438,Net per capita PIN (base 1999-2001),1965.0,Int. $,335.0,A,agriculture_pin
3,Occupied Palestinian Territory,434,Grs per capita PIN (base 1999-2001),1964.0,Int. $,0.0,A,non_food_pin
4,"Tanzania, United Republic of",438,Net per capita PIN (base 1999-2001),1995.0,Int. $,105.0,A,livestock_pin


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231078 entries, 0 to 233724
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Region            231078 non-null  object 
 1   Element Code      231078 non-null  object 
 2   Element           231078 non-null  object 
 3   Year              231078 non-null  float64
 4   Unit              231078 non-null  object 
 5   Dollar Amount     231078 non-null  float64
 6   Category          231078 non-null  object 
 7   Gross/Net Binary  231078 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 15.9+ MB


In [5]:
df.loc[df['Element'].isna()]

,Region,Element Code,Element,Year,Unit,Dollar Amount,Value Footnotes,Category
36730,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,livestock_pin
41795,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,non_food_pin
86401,A,"May include official, semi-official or estimat...",NaN,NaN,NaN,NaN,NaN,non_food_pin
88414,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,crops_pin
109768,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,agriculture_pin
113185,A,"May include official, semi-official or estimat...",NaN,NaN,NaN,NaN,NaN,livestock_pin
160684,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,food_pin
200497,A,"May include official, semi-official or estimat...",NaN,NaN,NaN,NaN,NaN,food_pin
227575,A,"May include official, semi-official or estimat...",NaN,NaN,NaN,NaN,NaN,agriculture_pin


In [6]:
# There are 2,638 values in the Dollar Amount Column that are zero
print(df.loc[df['Dollar Amount'] == 0.0]['Dollar Amount'].count())
print((2638/df.shape[0])*100, "%")

2638
1.1286768638357043 %


In [7]:
print('Element: ', df['Element'].unique())
print('-'*80)
print('Year: ', df['Year'].unique())
print('-'*80)
print('Unit: ', df['Unit'].unique())
print('-'*80)
print('Dollar Amount: ', df['Dollar Amount'].unique())
print('-'*80)
print('Value Footnotes: ', df['Value Footnotes'].unique())

Element:  ['Net per capita PIN (base 1999-2001)'
 'Grs per capita PIN (base 1999-2001)' 'Net PIN (base 1999-2001)'
 'Gross PIN (base 1999-2001)' 'Net Production 1999-2001 (1000 I$)'
 'Gross Production 1999-2001 (1000 I$)' nan]
--------------------------------------------------------------------------------
Year:  [1977. 1980. 1965. 1964. 1995. 1978. 1975. 1972. 1963. 1998. 1971. 1983.
 1988. 1999. 1976. 2007. 1973. 2006. 1967. 1962. 1994. 2001. 1987. 1984.
 1974. 1966. 2003. 2005. 1982. 1969. 1991. 1989. 1996. 2002. 1986. 1993.
 1961. 1981. 1990. 1997. 1992. 2000. 1970. 2004. 1968. 1979. 1985.   nan]
--------------------------------------------------------------------------------
Unit:  ['Int. $' '1000 Int. $' nan]
--------------------------------------------------------------------------------
Dollar Amount:  [1.150000e+02 1.020000e+02 3.350000e+02 ... 3.079169e+06 3.336800e+04
 2.411800e+04]
--------------------------------------------------------------------------------
Value Footno

# Start Cleaning

### Annotations about cleaning function

##### Element
- Seperate into Gross/Net
- Create Binary col based off of this

##### Year
- Replace with median value

##### Unit
- Replace with Int.

##### Value
- Since this is the information that we will be predicting on the test set, we are going to drop any row of the train set that contains a null or zero.

##### Value Footnotes
- Since all values are the same, we are going to drop this column. As it will have no sway on the model. 

## Handling Nulls and Dropping Data

In [8]:
# Drop Value Footnotes
df.drop(columns='Value Footnotes', inplace=True)

In [9]:
# Replacing null Year values with median (which is 1985), Element values with Gross, and Unit values with 'Int. $'
# Just in case; but it appears that any row with these values missing (in this DF) are also missing Dollar Amounts

year_med = st.median(df['Year'])
values = {'Year': year_med, 'Element': 'Gross', 'Unit': 'Int. $'}
df.fillna(value=values, inplace=True)

In [10]:
df.isna().sum()

Region           0
Element Code     0
Element          0
Year             0
Unit             0
Dollar Amount    9
Category         0
dtype: int64

In [11]:
# Drop rows containing nulls in the Dollar Amount col
df.dropna(subset=['Dollar Amount'], inplace=True)

# Drop rows containing zero in the Dollar Amount col
df = df[df['Dollar Amount'] != 0]

# Sanity Check
print('# of rows dropped: ', df_shape_og[0] - df.shape[0])

# of rows dropped:  2647


## Handling the 'Element' Column

In [12]:
element_col = []
binary_col = []
for val in df['Element']:
    if 'Net' in val:
        element_col.append('Net')
        binary_col.append(0)
    elif ('Gross') in val:
        element_col.append('Gross')
        binary_col.append(1)
    elif ('Grs') in val:
        element_col.append('Gross')
        binary_col.append(1)
    else:
        element_col.append('Gross')
        binary_col.append(1)

In [13]:
df['Element'], df['Gross/Net Binary'] = element_col, binary_col
df.head()

,Region,Element Code,Element,Year,Unit,Dollar Amount,Category,Gross/Net Binary
0,Caribbean +,438,Net,1977.0,Int. $,115.0,livestock_pin,0
1,Kiribati,434,Gross,1980.0,Int. $,102.0,crops_pin,1
2,Micronesia +,438,Net,1965.0,Int. $,335.0,agriculture_pin,0
4,"Tanzania, United Republic of",438,Net,1995.0,Int. $,105.0,livestock_pin,0
5,Albania,434,Gross,1978.0,Int. $,51.0,livestock_pin,1


In [14]:
## maybe replace unit with 1 for 'Int. $' and 1000 for '1000 Int. $' and multiplying dollar amount by unit then dropping col?
# Need to drop '+' in names
# Change dollar amount and year to int